In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

# Input and Output Directories
input_dir = "/Users/champakjyotikonwar/CODING/PROJECTS/MARS/calibrated"
output_dir = "/Users/champakjyotikonwar/CODING/PROJECTS/MARS/Result"
csv_output_path = "/Users/champakjyotikonwar/CODING/PROJECTS/MARS/spectrum_analysis_results.csv"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# List to store results
results = []

# Function to Convert Image to Grayscale and Resize to 64x64
def convert_to_grayscale_and_save(input_path, output_path, target_size=(64, 64)):
    img = cv2.imread(input_path)
    if img is None:
        print(f"Error: Unable to load image {input_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Resize to 64x64
    gray_resized = cv2.resize(gray, target_size)

    cv2.imwrite(output_path, gray_resized)
    return gray_resized

# Compute Fourier Transform (FFT)
def compute_fft(image):
    fft_image = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)
    fft_shifted = np.fft.fftshift(fft_image)
    magnitude_spectrum = 20 * np.log(cv2.magnitude(fft_shifted[:, :, 0], fft_shifted[:, :, 1]) + 1)
    return magnitude_spectrum

# Feature 1: FDMA Feature (Mean of Spectrum)
def ms_fdma_feature(spectrum):
    return np.mean(spectrum)

# Feature 2: Symmetry Feature (X & Y)
def ms_symmetry_feature(spectrum):
    center = spectrum.shape[0] // 2
    upper = spectrum[:center, :]
    lower = np.flipud(spectrum[center:, :])
    sym_x = np.mean(np.abs(upper - lower))
    
    left = spectrum[:, :center]
    right = np.fliplr(spectrum[:, center:])
    sym_y = np.mean(np.abs(left - right))
    
    return sym_x, sym_y

# Feature 3: Amplitude Moments (Mean, Variance, Skewness, Kurtosis)
def ms_amplitude_moment(spectrum):
    spectrum_flat = spectrum.flatten()  # Flatten to 1D array

    mean = np.mean(spectrum_flat)
    variance = np.var(spectrum_flat)
    skewness = skew(spectrum_flat)  # Measures asymmetry
    kurt = kurtosis(spectrum_flat)  # Measures sharp variations
    
    return mean, variance, skewness, kurt

# Process All Images in Input Directory
for filename in os.listdir(input_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):  # Process only image files
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        try:
            # Convert to Grayscale and Resize
            gray_image = convert_to_grayscale_and_save(input_path, output_path)
            if gray_image is None:
                continue  # Skip if image could not be processed

            # Compute FFT
            fft_magnitude = compute_fft(gray_image)

            # Compute Spectrum Analysis Features
            fdma = ms_fdma_feature(fft_magnitude)
            sym_x, sym_y = ms_symmetry_feature(fft_magnitude)
            amp_mean, amp_variance, amp_skewness, amp_kurtosis = ms_amplitude_moment(fft_magnitude)

            # Save Results
            results.append([filename, fdma, sym_x, sym_y, amp_mean, amp_variance, amp_skewness, amp_kurtosis])
            print(f"Processed {filename}: FDMA = {fdma:.4f}, Sym_X = {sym_x:.4f}, Sym_Y = {sym_y:.4f}, Amp_Mean = {amp_mean:.4f}, Amp_Var = {amp_variance:.4f}, Skewness = {amp_skewness:.4f}, Kurtosis = {amp_kurtosis:.4f}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Save results to CSV
df = pd.DataFrame(results, columns=["Filename", "MFDMA", "Symmetry_X", "Symmetry_Y", "Amp_Mean", "Amp_Variance", "Amp_Skewness", "Amp_Kurtosis"])
df.to_csv(csv_output_path, index=False)

print(f"\nProcessing completed. Results saved to: {csv_output_path}")


Processed 0042MR0001940470102262E01_DRCL.JPG: FDMA = 108.4478, Sym_X = 13.4372, Sym_Y = 13.3934, Amp_Mean = 108.4478, Amp_Var = 216.8656, Skewness = -0.0667, Kurtosis = 4.1608
Processed 0154ML0008510220104513I01_DRCL.JPG: FDMA = 122.8400, Sym_X = 14.1300, Sym_Y = 13.8480, Amp_Mean = 122.8400, Amp_Var = 407.0704, Skewness = 0.3331, Kurtosis = 1.2101
Processed 0039MR0001770030101222E01_DRCL.JPG: FDMA = 127.3090, Sym_X = 13.9753, Sym_Y = 13.8872, Amp_Mean = 127.3090, Amp_Var = 262.9887, Skewness = 0.0071, Kurtosis = 2.7863
Processed 0178ML0006460000105248E01_DRCL.JPG: FDMA = 135.6312, Sym_X = 14.4592, Sym_Y = 14.0752, Amp_Mean = 135.6312, Amp_Var = 317.8813, Skewness = 0.2860, Kurtosis = 2.6486
Processed 0044MR0002011240102503E01_DRCL.JPG: FDMA = 125.4495, Sym_X = 13.7213, Sym_Y = 14.0055, Amp_Mean = 125.4495, Amp_Var = 174.2827, Skewness = -0.7445, Kurtosis = 5.0906
Processed 0072MR0005610060103631E01_DRCL.JPG: FDMA = 138.0712, Sym_X = 14.2051, Sym_Y = 14.8987, Amp_Mean = 138.0712, Amp_V